<a href="https://colab.research.google.com/github/mmosc/ppp/blob/main/Petrol_price_prediction_Women_Hackathon_Linz_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Petrol Price Prediction
## Introduction 
This tutorial is meant to be an introduction on how to use [Facebook Prophet](https://facebook.github.io/prophet/) for forecasting.

The tutorial is a follow-up of the project `Peep-peep-peep` developed by Larisa Manciu, Ariona Mahmutaj, ??? ???, and Marta Moscati for the first [Female Coders Hackathon](https://www.meetup.com/female-coders-linz/events/286136444/).

Many thanks to the mentors - Magdalena Radinger in particular - for their support in developing the project!
## Problem definition 

We would like to model the evolution of fuel prices.

First things first, we need to collect, clean, and organize the data. Luckily for us, the UK government provides data on the evolution of fuel and diesel prices in their country, therefore - and for the sake of simplicity - we will focus on UK for this tutorial. 

Having a look at [their website](https://www.data.gov.uk/dataset/c174a981-b0f2-4b39-adc0-1d0a27a7d8c9/petrol-and-diesel-prices), we see that prices are recorded weekly from 2003 till today (12 September 2022 at the time of writing and downloading the data), and this is the time-span we will focus on. 

## Datased
### Disclaimer
I purposely did NOT include screenshots or embeddings of the data when referring to them in the next section, since I want you to explore them yourself :) . So I included links, instead.

### Get familiar with the data
Hm... at first it looks as if we had to do some crawling, since each week is given in a different file... let's check the links to see what they look like, let's take for instance [30. March 2020](https://www.gov.uk/government/statistics/weekly-road-fuel-prices). 

Wow, we were luckier than we thought: apparently, this redirects us to a page containing data from ALL years in a single file! Awesome!

Let's get the [`.csv`](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1103998/CSV_120922.csv) (because we love `.csv`'s...) and first look at it with Excel, LibreOffice, or a similar [spreadsheet](https://docs.google.com/spreadsheets/d/1XBEkUu4fMXi91S8Z5fb47OGm0S0sz9pc64Vq4c3P2ew/edit?usp=sharing). Make sure to select the comma as separator, if things look horrible (that's what [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) stands for, after all...). 

So, what are we seeing there? The headers already give us some information: We have the "Pump price in pence/liter", i.e. the price at which the fuel is sold, the "Duty rate in pence/liter", and the "VAT percentage rate". Some googling tells us that both the Duty rate and the VAT are taxes, and that the pump price is brutto. For the sake of simplicity, we will restrict to the prediction of brutto prices in this tutorial, and leave the task of the netto prediction as an exercise.

The header was not enough though, and we still could not figure out what those acronyms stand for...together with Google, the other biggest source of information is the documentation. We can get more information if we look at the [excel version of the dataset](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1103996/Weekly_Fuel_Prices_120922.xlsx). The highlights sheet tells us that ULSP stands for Petrol, while ULSD stands for Diesel. So we will restrict to the ULSP-related columns for this project. 

Okay, now that we understand how our dataset looks like, we are ready to start coding!

## A closer look at the data
Let's get a closer look at the data with the swiss knife of data science - pandas. Pandas allows to read .csv files with a simple one-line command. 

In [2]:
import pandas as pd

Substitute this with the path to the `.csv` file.

In [7]:
data_path = '/content/drive/MyDrive/weekly_fuel_prices.csv'
prices_df = pd.read_csv(data_path)

UnicodeDecodeError: ignored

Hmm...something went wrong there. Apparently, some character was not recognized! By googling "byte 0xa3" we find out that the problem is the £...so...well, let just try to get rid of it and reload the file. (Be smart there, and use "find and substitute").

In [11]:
data_path_no_pound = '/content/drive/MyDrive/weekly_fuel_prices_no_pound.csv'
prices_df = pd.read_csv(data_path_no_pound)

Awesome, that worked! Let's inspect the file. Pandas loads `.csv` files as a DataFrame, which is something between a table and a database, as we will see.

In [12]:
type(prices_df)

pandas.core.frame.DataFrame

The DataFrame object has a method called `.head()`, which is very similar to the `bash` command `head`, and displays the first lines of the DataFrame. Let's try this out.

In [13]:
prices_df.head()

,Weekly Prices time series,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Weekly Prices time series,Pump price in pence/litre,NaN,Duty rate in pence/litre,NaN,VAT percentage rate,NaN,NaN,ULSP = Ultra low sulphur unleaded petrol,NaN
1,Date,ULSP,ULSD,ULSP,ULSD,ULSP,ULSD,NaN,ULSD = Ultra low sulphur diesel,NaN
2,09/06/2003,74.59.00,77.17.00,46.22.00,46.22.00,17.5,17.5,NaN,NaN,NaN
3,16/06/2003,74.47.00,77.09.00,46.22.00,46.22.00,17.5,17.5,NaN,ULSP and ULSD,NaN
4,23/06/2003,74.42.00,77.02.00,46.22.00,46.22.00,17.5,17.5,NaN,Duty rate per litre () from 7 March 2001,76.22.00
